data download : https://arena.kakao.com/c/2/data

In [1]:
from glob import glob
from itertools import chain
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
from torchvision import transforms
from torchvision import models
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.optim import lr_scheduler
from torch import optim
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
import time
%matplotlib inline

In [2]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, 
                              DataLoader,
                              TensorDataset)
import tqdm
from sklearn import model_selection

In [3]:
#시드 고정
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)

In [4]:
data = pd.read_hdf('merge_data_real.h5',key="df")

In [5]:
data.head()

,dt,hr,user_id,article_id,score,author_id,following_list,keyword_list
0,20181001,00,#e208be4ffea19b1ceb5cea2e3c4dc32c,@kty0613_91,5,@kty0613,"[@kecologist, @darunwrite, @investup, @travie,...",[]
1,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@miamiyoung_31,2,@miamiyoung,"[@hyunjooje, @posselavaboy, @mupasa324, @minse...","[{'cnt': 1, 'keyword': 'DDU 조건'}, {'cnt': 1, '..."
2,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@banksalad_49,2,@banksalad,"[@hyunjooje, @posselavaboy, @mupasa324, @minse...","[{'cnt': 1, 'keyword': 'DDU 조건'}, {'cnt': 1, '..."
3,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@rlfrjsdn_95,2,@rlfrjsdn,"[@hyunjooje, @posselavaboy, @mupasa324, @minse...","[{'cnt': 1, 'keyword': 'DDU 조건'}, {'cnt': 1, '..."
4,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@readme999_140,2,@readme999,"[@hyunjooje, @posselavaboy, @mupasa324, @minse...","[{'cnt': 1, 'keyword': 'DDU 조건'}, {'cnt': 1, '..."


In [6]:
train_data = data.drop(["dt","hr",'author_id','following_list','keyword_list'],axis=1)

In [15]:
train_data.head()

,user_id,article_id,score,user_id_int,article_id_int
0,#e208be4ffea19b1ceb5cea2e3c4dc32c,@kty0613_91,5,0,0
1,#0a3d493f3b2318be80f391eaa00bfd1c,@miamiyoung_31,2,1,1
2,#0a3d493f3b2318be80f391eaa00bfd1c,@banksalad_49,2,1,2
3,#0a3d493f3b2318be80f391eaa00bfd1c,@rlfrjsdn_95,2,1,3
4,#0a3d493f3b2318be80f391eaa00bfd1c,@readme999_140,2,1,4


user_id와 article_id가 string이라서 torch.tensor의 dtype을 지정할 수가 없다. 매핑시키면서 딕셔너리까지 만들어서 저장해두고 쓰자.

In [16]:
user_dic = {}
article_dic = {}

In [17]:
def id_dic_mapper(df):
    user_unique_list = df['user_id'].unique().tolist()
    articel_uniqe_list = df['article_id'].unique().tolist()
    for idx,row in enumerate(user_unique_list):
        user_dic.update({row:idx})
    for idx,row in enumerate(articel_uniqe_list):
        article_dic.update({row:idx})        

In [18]:
id_dic_mapper(train_data)

In [19]:
train_data['user_id_int'] = train_data['user_id'].map(user_dic)
train_data['article_id_int'] = train_data['article_id'].map(article_dic)

In [20]:
train_data.head()

,user_id,article_id,score,user_id_int,article_id_int
0,#e208be4ffea19b1ceb5cea2e3c4dc32c,@kty0613_91,5,0,0
1,#0a3d493f3b2318be80f391eaa00bfd1c,@miamiyoung_31,2,1,1
2,#0a3d493f3b2318be80f391eaa00bfd1c,@banksalad_49,2,1,2
3,#0a3d493f3b2318be80f391eaa00bfd1c,@rlfrjsdn_95,2,1,3
4,#0a3d493f3b2318be80f391eaa00bfd1c,@readme999_140,2,1,4


In [21]:
train_data["article_id_int"].max()

505840

In [22]:
train_user = pd.read_csv('data/dev_users.txt',names=["user_id"])

In [23]:
train_user['user_id_int'] = train_user['user_id'].map(user_dic)

In [24]:
train_user.head()

,user_id,user_id_int
0,#d6866a498157771069fdf15361cb012b,282354
1,#f963fb8c5d9d14d503fc4e80bd8617b4,31643
2,#87a6479c91e4276374378f1d28eb307c,253506
3,#677e984e245b344f61dc5d3cc1f352c8,170668
4,#519f45eb14e4807e8714fb7e835463eb,150484


아래로 확인해본 결과 예측할 유저들의 목록에 train에 없는 유저(즉, 신규유저)는 없다. 그대로 train모델을 적용시켜서 얻어내자

In [17]:
train_user.isna().sum()

user_id        0
user_id_int    0
dtype: int64

In [25]:
X = train_data[["user_id_int","article_id_int"]].values
Y = train_data[["score"]].values

train_X, test_X, train_Y, test_Y = model_selection.train_test_split(X,Y, test_size=0.2,random_state=42)

train_dataset = TensorDataset(
                                torch.tensor(train_X, dtype=torch.int64), torch.tensor(train_Y, dtype=torch.float64)
                            )
test_dataset = TensorDataset(
                                torch.tensor(test_X, dtype=torch.int64), torch.tensor(test_Y, dtype=torch.float64)
                            )

In [26]:
train_loader = DataLoader(
                            train_dataset, batch_size=256, num_workers=4, shuffle=True
                            )
test_loader = DataLoader(
                            test_dataset, batch_size=256, num_workers=4
                            )

neural_MF정의하고

In [21]:
class MatrixFactorization(nn.Module):
    def __init__(self, max_user, max_item, k=20):
        super().__init__()
        self.max_user = max_user
        self.max_item = max_item
        self.user_emb = nn.Embedding(max_user, k, 0)
        self.user_emb.weight.requires_grad=True
        self.item_emb = nn.Embedding(max_item, k, 0)
        self.item_emb.weight.requires_grad=True
        
    def forward(self, x):
        user_idx = x[:, 0]
        item_idx = x[:, 1]
        user_feature = self.user_emb(user_idx)

        item_feature = self.item_emb(item_idx)

        
        # user_feature*item_feature는 (batch_size,k) 차원이므로
        # k의 sum을 구하면 각 샘플의 내적이 된다
        out = torch.sum(user_feature * item_feature, 1)

        
        # [0, 3] 범위 내로 조정
        out = nn.functional.sigmoid(out) * 5

        return out

In [27]:
class NeuralMatrixFactorization(nn.Module):
    def __init__(self, max_user, max_item,
                 user_k=15, item_k=15,
                 hidden_dim=50):
        super().__init__()
        self.user_emb = nn.Embedding(max_user, user_k, 0)
        self.user_emb.weight.requires_grad=True
        self.item_emb = nn.Embedding(max_item, item_k, 0)
        self.item_emb.weight.requires_grad=True
        self.mlp = nn.Sequential(
            nn.Linear(user_k + item_k, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(p=0.5),
            nn.Linear(hidden_dim, 1)
        )
            
    def forward(self, x):
        user_idx = x[:, 0]
        item_idx = x[:, 1]
        user_feature = self.user_emb(user_idx)
        item_feature = self.item_emb(item_idx)
        # 사용자 특이량과 상품 특이량을 모아서 하나의 벡터로 만듦
        out = torch.cat([user_feature, item_feature], 1)
        # 모은 벡터를 MLP에 넣는다
        out = self.mlp(out)
        out = nn.functional.sigmoid(out) * 5
        return out.squeeze()

net 정의해주고

In [28]:
max_user, max_item = X.max(0)
# np.int64형을 파이썬의 표준 int로 캐스트
max_user = int(max_user)
max_item = int(max_item)
# net = MatrixFactorization(max_user+1, max_item+1)
net = NeuralMatrixFactorization(max_user+1, max_item+1)

In [21]:
max_user

306221

In [22]:
len(train_data["user_id_int"].unique())

306222

In [23]:
max_item

505840

In [24]:
len(train_data["article_id_int"].unique())

505841

평가를 위한 함수 정의

In [29]:
def eval_net(net, loader, score_fn=nn.functional.
l1_loss, device="cpu"):
    ys = []
    ypreds = []
    for x, y in loader:
        x = Variable(x).to(device)
        ys.append(y)
        with torch.no_grad():
            ypred = net(x).to("cpu").view(-1).double()
        ypreds.append(ypred)
    score = score_fn(torch.cat(ys).squeeze(), 
torch.cat(ypreds))
    return score.item()

In [30]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [31]:
from statistics import mean

net.to("cuda:0")
opt = optim.Adam(net.parameters(), lr=0.001)
loss_f = nn.MSELoss()

for epoch in range(7):
    loss_log = []
    for x, y in tqdm.tqdm(train_loader):
        x = Variable(x).to("cuda:0")
        y = Variable(y).to("cuda:0")
        o = net(x)


        loss = loss_f(o, y.view(-1).float())


        net.zero_grad()
        loss.backward()
        opt.step()
        loss_log.append(loss.item())
    test_score = eval_net(net, test_loader, device="cuda:0")
    print(epoch, mean(loss_log), test_score, flush=True)

  0%|                                                                                        | 0/44816 [00:00<?, ?it/s]C:\Users\zone\Anaconda3\lib\site-packages\torch\nn\functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|████████████████████████████████████████████████████████████████████████████| 44816/44816 [19:41<00:00, 31.82it/s]


0 1.2290313505500778 0.6883777247837986


100%|████████████████████████████████████████████████████████████████████████████| 44816/44816 [18:46<00:00, 39.77it/s]


1 0.9581540376692744 0.6481205000256044


100%|████████████████████████████████████████████████████████████████████████████| 44816/44816 [17:56<00:00, 41.61it/s]


...

4 0.8561072174854701 0.6156874020763531


100%|████████████████████████████████████████████████████████████████████████████| 44816/44816 [35:02<00:00, 21.32it/s]


5 0.8387115625195771 0.6074555697807316


100%|████████████████████████████████████████████████████████████████████████████| 44816/44816 [24:35<00:00, 30.36it/s]


6 0.8224124994038503 0.5979279755594994


In [37]:
article_dic_swap = {val:key for (key, val) in article_dic.items()}

In [27]:
article_dic_swap

{0: '@kty0613_91',
 1: '@miamiyoung_31',
 2: '@banksalad_49',
 3: '@rlfrjsdn_95',
 4: '@readme999_140',
 5: '@jordan777_1558',
 6: '@charlieoppa86_33',
 7: '@supims_189',
 8: '@jericho27_385',
 9: '@charlieoppa86_39',
 10: '@sosoceo_72',
 11: '@thankyousomuch_74',
 12: '@realcast_271',
 13: '@joeunha_4',
 14: '@yoonvi_3',
 15: '@hyejinchoi_122',
 16: '@hyejinchoi_86',
 17: '@hyejinchoi_42',
 18: '@hyejinchoi_13',
 19: '@hyejinchoi_21',
 20: '@hyejinchoi_7',
 21: '@hyejinchoi_75',
 22: '@hyejinchoi_88',
 23: '@hyejinchoi_112',
 24: '@doyeonsunim_240',
 25: '@k52524_297',
 26: '@bule13_33',
 27: '@kwong7_70',
 28: '@oasisoffice_6',
 29: '@hajunho_334',
 30: '@eundang_472',
 31: '@bookguru_29',
 32: '@namulab_20',
 33: '@bestkkim_45',
 34: '@toosim_172',
 35: '@leejarak_209',
 36: '@nplusu_49',
 37: '@fromsirin_31',
 38: '@lifeplus_150',
 39: '@hotelscomkr_446',
 40: '@nana4316_114',
 41: '@allstay_981',
 42: '@asi_389',
 43: '@elang8151_154',
 44: '@7-words_12',
 45: '@bokee_24',
 46: '#

answering함수를 통해 ---> 추천하는 상위 100개의 아이템 id를 가져온다. 이때 이미 본 아이템을 제외해주는 과정을 거치게 된다.

In [38]:
already_watch_df = train_data[["user_id_int","article_id_int"]]
already_watch_df.head()

,user_id_int,article_id_int
0,0,0
1,1,1
2,1,2
3,1,3
4,1,4


In [39]:
item_li = []
for i in range(max_item+1):
    item_li.append(i)
item_set = set(item_li)


In [40]:
def answering(df):
    #추천 아이템을 얻고자하는 유저의 아이디 획득
    user_id_int = df["user_id_int"]
    #그 유저가 이미 본 article은 추천하지 말아야한다. 따라서 이미 본(already_watch라고 표현) article들 목록 작성
    already_watch_list = already_watch_df[already_watch_df["user_id_int"]==user_id_int]["article_id_int"].tolist()
    #리스트로 차집합을 구하는건 오래걸리니까 set으로 만들어서 차집합을 구하고
    non_watch_set = item_set-set(already_watch_list)
    #다시 집합을 리스트로 바꿔주고 --> 이렇게 구한 리스트는 해당하는 유저가 아직 안본 글들의 모음이다(non_watch)
    non_watch_list = list(non_watch_set)
    #아래의 for문을 통해서 아직 안본 글들에 해당하는 [유저아이디, 아이템아이디]를 가지는 리스트를 만들어준다
    non_watch_query = []
    for non_watch_idx in non_watch_list:
        non_watch_query.append([user_id_int,non_watch_idx])
    #만든 리스트를 텐서로 바꿔준다
    query = torch.tensor(non_watch_query).long()
    #쿼리를 net에 보내서 탑100의 스코어와 인덱스를 얻어낸다.
    # 첫번째 리턴은 score를 리턴하지만 지금 필요한건 indices이므로 _로 처리
    _, indices = torch.topk(net(query.to("cuda:0")), 100)
    return indices
    #for문을 통해 100개의 추천 article id를 가지는 df를 만들어낸다.
#     for idx, article_id in enumerate(indices.tolist()):
#         df['{}'.format(idx)] = article_id
#     return df

answering함수를 적용시켜서 각 유저들의 상위 100가지의 추천 article_int 정보를 담고있는 df을 생성한다. 함수 적용이 오래걸린다 ---> 3000건에 적용시키는데 52min 37s 걸림

In [41]:
df = pd.DataFrame()
df

""


In [42]:
for i in tqdm.tqdm(train_user.apply(answering,axis=1)):
    c = i.cpu()
    s = pd.Series(c)
    df = df.append(s,ignore_index=True)

100%|█████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:05<00:00, 525.44it/s]


In [43]:
df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,403120.0,402699.0,418194.0,422377.0,312074.0,302919.0,313364.0,316549.0,250901.0,241824.0,...,6105.0,6374.0,13962.0,12156.0,14064.0,14983.0,88247.0,83102.0,69722.0,80142.0
1,457485.0,448861.0,459907.0,465712.0,95555.0,82308.0,95556.0,109213.0,70932.0,70931.0,...,3191.0,5437.0,6808.0,6795.0,7785.0,8040.0,40733.0,39731.0,37843.0,37844.0
2,423120.0,422399.0,425832.0,431811.0,277486.0,254815.0,302926.0,306893.0,163683.0,131841.0,...,2991.0,5440.0,6812.0,6799.0,7791.0,10422.0,44653.0,44281.0,42043.0,43437.0
3,58727.0,57935.0,58772.0,61788.0,40552.0,38979.0,40741.0,40742.0,34114.0,34026.0,...,1774.0,1998.0,3054.0,2868.0,3159.0,3194.0,21432.0,21335.0,19727.0,19728.0
4,254739.0,252380.0,258314.0,260602.0,116204.0,96676.0,125055.0,127326.0,83046.0,82259.0,...,2499.0,2852.0,7767.0,6789.0,8648.0,10384.0,46501.0,45357.0,40688.0,41252.0
5,268085.0,267185.0,273968.0,284740.0,83107.0,73685.0,91362.0,95567.0,70743.0,68144.0,...,2868.0,2991.0,6812.0,6799.0,10077.0,10422.0,37851.0,35044.0,33633.0,35043.0
6,331852.0,326822.0,345086.0,350533.0,91359.0,91357.0,91361.0,95564.0,70940.0,70939.0,...,5437.0,6103.0,10074.0,8043.0,10419.0,11127.0,37125.0,35041.0,33629.0,35040.0
7,122737.0,104615.0,125092.0,127363.0,57898.0,57250.0,58690.0,61179.0,46522.0,45377.0,...,1764.0,1984.0,5415.0,5414.0,6077.0,6782.0,30610.0,30004.0,29859.0,29860.0
8,290204.0,274359.0,291274.0,291349.0,127321.0,116202.0,131832.0,134028.0,83043.0,80093.0,...,1409.0,1758.0,6084.0,2851.0,6788.0,10391.0,44601.0,41255.0,40693.0,41254.0
9,228648.0,227232.0,232051.0,238743.0,79721.0,73685.0,80149.0,82320.0,70942.0,70743.0,...,2521.0,2868.0,6812.0,6799.0,10077.0,10422.0,40742.0,40741.0,37851.0,38615.0


In [44]:
article_number =  df.copy()

In [37]:
article_number

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,484429.0,479018.0,473632.0,474231.0,448345.0,444491.0,448883.0,451574.0,383796.0,358987.0,...,442583.0,456321.0,412177.0,393426.0,423461.0,23285.0,363713.0,95695.0,31977.0,47545.0
1,503734.0,391206.0,448861.0,490253.0,438163.0,22186.0,33028.0,35035.0,40734.0,40733.0,...,31401.0,128163.0,409062.0,467941.0,372959.0,440622.0,483976.0,15306.0,346956.0,406741.0
2,70270.0,68143.0,70753.0,70941.0,41305.0,40743.0,42043.0,43200.0,39739.0,37852.0,...,2521.0,2868.0,4755.0,4598.0,4756.0,5440.0,21335.0,21229.0,20719.0,20720.0
3,496836.0,495581.0,479053.0,493991.0,448881.0,448798.0,450092.0,451574.0,434631.0,423772.0,...,57450.0,70271.0,91360.0,73685.0,161396.0,19977.0,72226.0,42043.0,19726.0,30648.0
4,350448.0,336650.0,270102.0,306518.0,233319.0,184721.0,250832.0,254739.0,73627.0,70885.0,...,2852.0,3178.0,6776.0,6084.0,6789.0,45357.0,3145.0,40757.0,70213.0,71733.0
5,391246.0,385885.0,350537.0,358699.0,134111.0,179308.0,184797.0,250909.0,448901.0,440926.0,...,457525.0,403131.0,493629.0,428862.0,491028.0,32993.0,11738.0,29895.0,354484.0,419329.0
6,505412.0,391241.0,361132.0,362962.0,350533.0,331852.0,358390.0,358695.0,184794.0,163792.0,...,3191.0,5438.0,10419.0,8679.0,10495.0,11127.0,45409.0,16982.0,6796.0,6809.0
7,505028.0,438124.0,358623.0,254765.0,448822.0,450042.0,490206.0,498976.0,19694.0,18996.0,...,31412.0,419154.0,441918.0,498596.0,221906.0,23922.0,450832.0,454862.0,437994.0,376562.0
8,503571.0,254723.0,250817.0,184713.0,70884.0,70883.0,68087.0,95500.0,70213.0,391120.0,...,505245.0,91292.0,817.0,475153.0,5757.0,408952.0,452027.0,41991.0,30602.0,435418.0
9,425172.0,423683.0,426353.0,438200.0,273619.0,252439.0,280483.0,284552.0,184797.0,180043.0,...,3194.0,5440.0,10077.0,8682.0,10422.0,10498.0,65063.0,57286.0,53442.0,54890.0


In [45]:
user_article_number = train_user.join(article_number)

In [46]:
user_article_number.head()

,user_id,user_id_int,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
0,#d6866a498157771069fdf15361cb012b,282354,403120.0,402699.0,418194.0,422377.0,312074.0,302919.0,313364.0,316549.0,...,6105.0,6374.0,13962.0,12156.0,14064.0,14983.0,88247.0,83102.0,69722.0,80142.0
1,#f963fb8c5d9d14d503fc4e80bd8617b4,31643,457485.0,448861.0,459907.0,465712.0,95555.0,82308.0,95556.0,109213.0,...,3191.0,5437.0,6808.0,6795.0,7785.0,8040.0,40733.0,39731.0,37843.0,37844.0
2,#87a6479c91e4276374378f1d28eb307c,253506,423120.0,422399.0,425832.0,431811.0,277486.0,254815.0,302926.0,306893.0,...,2991.0,5440.0,6812.0,6799.0,7791.0,10422.0,44653.0,44281.0,42043.0,43437.0
3,#677e984e245b344f61dc5d3cc1f352c8,170668,58727.0,57935.0,58772.0,61788.0,40552.0,38979.0,40741.0,40742.0,...,1774.0,1998.0,3054.0,2868.0,3159.0,3194.0,21432.0,21335.0,19727.0,19728.0
4,#519f45eb14e4807e8714fb7e835463eb,150484,254739.0,252380.0,258314.0,260602.0,116204.0,96676.0,125055.0,127326.0,...,2499.0,2852.0,7767.0,6789.0,8648.0,10384.0,46501.0,45357.0,40688.0,41252.0


In [47]:
recommend_df_copy = user_article_number.drop(["user_id_int"],axis=1)

In [48]:
recommend_df_copy.head()

,user_id,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,#d6866a498157771069fdf15361cb012b,403120.0,402699.0,418194.0,422377.0,312074.0,302919.0,313364.0,316549.0,250901.0,...,6105.0,6374.0,13962.0,12156.0,14064.0,14983.0,88247.0,83102.0,69722.0,80142.0
1,#f963fb8c5d9d14d503fc4e80bd8617b4,457485.0,448861.0,459907.0,465712.0,95555.0,82308.0,95556.0,109213.0,70932.0,...,3191.0,5437.0,6808.0,6795.0,7785.0,8040.0,40733.0,39731.0,37843.0,37844.0
2,#87a6479c91e4276374378f1d28eb307c,423120.0,422399.0,425832.0,431811.0,277486.0,254815.0,302926.0,306893.0,163683.0,...,2991.0,5440.0,6812.0,6799.0,7791.0,10422.0,44653.0,44281.0,42043.0,43437.0
3,#677e984e245b344f61dc5d3cc1f352c8,58727.0,57935.0,58772.0,61788.0,40552.0,38979.0,40741.0,40742.0,34114.0,...,1774.0,1998.0,3054.0,2868.0,3159.0,3194.0,21432.0,21335.0,19727.0,19728.0
4,#519f45eb14e4807e8714fb7e835463eb,254739.0,252380.0,258314.0,260602.0,116204.0,96676.0,125055.0,127326.0,83046.0,...,2499.0,2852.0,7767.0,6789.0,8648.0,10384.0,46501.0,45357.0,40688.0,41252.0


In [49]:
for i in tqdm.tqdm(range(100)):
    recommend_df_copy[i] = recommend_df_copy[i].map(article_dic_swap)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:24<00:00,  1.46s/it]


In [50]:
recommend_df_copy

,user_id,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,#d6866a498157771069fdf15361cb012b,@goyohan_126,@jessicapark_2,@supims_481,@jjackjjackjjack_39,@dailylife_491,@yeatsjh_646,@heyhaeun_61,@cheesewinepang_31,@rhodia_140,...,@linkandleave_9,@varo_620,@ansyd_538,@inkkou_1,@movedbymovie_62,@dizzo_8,@jobhelping_229,@km0603_17,@yumileewyky_196,@hamtongz_6
1,#f963fb8c5d9d14d503fc4e80bd8617b4,@3fbaksghkrk_27,@skshin70_1,@haninorway19_59,@scottpark_20,@ithinkso91_7,@lieoflie_75,@parkdabin_470,@hans2cloud9_3,@jinhakjung3_226,...,@tenbody_1302,@dahyesong91_104,@superstrings_2,@blurwind_27,@hiwriting_257,@jordan777_2223,@pyossi_2,@myolivenote_719,@ysp0722_1671,@ysp0722_1617
2,#87a6479c91e4276374378f1d28eb307c,@jabkim77_12,@dooholee_73,@nonan_49,@eosinvestor_6,@waxingmoon_65,@brunch_75,@appleynh_37,@giantaj_85,@brunch8yx6_7,...,@merryseo_30,@brunch_119,@brunch_134,@brunch_11,@brunch_102,@brunch_79,@brunch_24,@brunch_64,@brunch_27,@brunch_122
3,#677e984e245b344f61dc5d3cc1f352c8,@lovebrander_27,@brunch_116,@masturno1_4,@brunch_107,@gddaddi_79,@insaboy_65,@brunch_29,@brunch_10,@creative_98,...,@brunch_139,@brunch_136,@byulpd_417,@brunch_138,@roysday_231,@brunch_137,@brunch_53,@brunch_103,@brunch_120,@brunch_97
4,#519f45eb14e4807e8714fb7e835463eb,@fashionlab_87,@tartar0_20,@doubleshot_596,@songyoungdae_12,@jordan777_1862,@sominlee_3,@harink_19,@edigna_27,@pinastro_9,...,@boboc_64,@entrench_69,@kimds1991_108,@gochunsoo_79,@ylangylang_74,@cola-gom_25,@mymykmcyobv_114,@hansalt58_164,@alicemelbourne_130,@varo_571
5,#ec0fb734ba02a29c62c64e7ac7a8f13e,@seonyoungwang_11,@topasvga_374,@younghakjang_95,@nasica_32,@ymoon_91,@brunch_140,@brunch_61,@brunch_98,@yeoulhan_1,...,@brunch_138,@merryseo_30,@brunch_134,@brunch_11,@redhairanne_98,@brunch_79,@brunch_127,@brunch_65,@brunch_20,@brunch_67
6,#a0df5bd0e5a5bbc28b87f8c64462667c,@minnation_840,@bookfit_1114,@junhocho_20,@yunachoi39_89,@brunch_89,@sweetmeen_90,@brunch_86,#a736fa9eac4e73a26da65976dbc70ab4_6,@jipyeongseon_10,...,@dahyesong91_104,@yonabr_156,@winebee_67,@jordan777_2220,@toeickers_27,@worldeconomy_4,@hawann_70,@eduinus_9,@kkbbtr_92,@bookfit_3115
7,#157f9687d223501f0e18cc469462cab1,@merry-2015_134,@y-siii_29,@noche_60,@jaemyungshin_2,@am327_21,@heroyw1_107,@am327_52,@tjrrkdi91_39,@shoesaw_27,...,@smoker3_45,@atommerce_1,@creatorj_76,@onlymisato_2,@ehqhdtksrlf5_83,@huiwon56_408,@sciforus_98,@cathongzo_62,@book-habit_27,@dalcomsky_49
8,#dfe07d49ef81ac7f00653b5b82b70d83,@bookguru_40,@daniiiii_43,@writeofmylife_121,@cosmos-j_295,@didgpdms821_119,@jordan777_1860,@jericho27_238,@yemaya_254,@realcast_318,...,@punytraveler_8,@hosslee_293,@tskim_1,@entrench_70,@hermite236_582,@matteroftaste_23,@subori30_21,@varo_547,@goodbrather_56,@varo_568
9,#dce6d42815d547618757cd3d963c112c,@kakao-it_299,@wikitree_2962,@dimension-value_44,@sunsutu_846,@juliakimcued_87,@brunch_140,@taekangk_10,@brunch_93,@brunch_112,...,@brunch_59,@brunch_138,@brunch_134,@brunch_11,@redhairanne_98,@brunch_79,@brunch_10,@brunch_29,@brunch_127,@mobiinside_1269


In [51]:
#결과 저장
recommend_df_copy.to_csv(r'recommend.txt',header=None, sep=' ', index=False)

In [99]:
recommend_df_copy

,user_id,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,#d6866a498157771069fdf15361cb012b,@botongmarketer_176,@architect-shlee_439,@cogitoergosum_2,@travelbook_295,@fishz_102,@lifeplus_200,@kirin1004_144,@sumgyul_35,@sting762_367,...,@goyohan_91,@expediakr_49,@citcoaching_34,@minariganari_22,@sukhwanheo_35,@derinna_40,@honeytip_675,@adipoman_289,@lglgh1983_17,@thecapitalist_54
1,#f963fb8c5d9d14d503fc4e80bd8617b4,@jaydenkim_21,@insateam_257,@2ivorybear_138,@lesreports_67,@ljs75600_74,@allstay_35,@arti-khan_9,@nplusu_173,@madame_2,...,@yejinchoi_21,@alvis_3,@sabjakoon_1,@seonkicheong_620,@kimsso17-18-1_3,@moktaksori_29,@realcast_53,@skylovedk17_48,@edith_23,@cid0630_61
2,#87a6479c91e4276374378f1d28eb307c,@onairpiano_8,@fromjin_15,@keunsori5_446,@architect-shlee_256,@edityou_184,@lwh0416_17,@lcbin_15,@dreamuniversity_12,@dajwj0516_87,...,@elara1020_263,@shengxia_24,@legancebaek_21,@profound29_56,@driver888_3,@jhyunwriter_400,@chosaeun_4,@merry-2015_45,@travel-bike_273,@ykimkim_29
3,#677e984e245b344f61dc5d3cc1f352c8,@parcchanyong_141,@fullssamh7ty_65,@piggest_29,@matilda_57,@puding_134,@echoi0816_6,@dosoonpark_58,@todaykangsein_31,@kosy0346_27,...,@onceaweek_50,@brik_37,@taejin107_1,@ginocchiodestro_34,@ost1219_127,@awakeup_8,@dnfladydgk_364,@connieworld_3,@leichin_60,@zyeun_12
4,#519f45eb14e4807e8714fb7e835463eb,@sofung70_29,@gwjun11111_14,@jlast119_31,@lkh3250_491,@eunbinbaek_4,@ggomma_46,@99rock_79,@keunsori5_611,@kimsawon_34,...,@moonyuai_11,@varo_273,@howyoung_3,@bluemoonwolf_32,@labonneheure_28,@hakgome_135,@angum4_72,@haklaekim_7,@jo2660_67,@fashionlab_96
5,#ec0fb734ba02a29c62c64e7ac7a8f13e,@younghurckc_195,@fromjin_12,@wlsalsdnek_14,@hosungko_149,@yiyeon-a_5,@rothem_151,@dnfladydgk_135,@baka_84,@chaserin0_5,...,@tokyodomin_381,@seasonalwriter_83,@skukyeo_93,@son6336_30,@seory_53,@ajtwlstmdgks_7,@bestkkim_47,@kingsofgreed_31,@shinhyea_2,@doranproject_87
6,#a0df5bd0e5a5bbc28b87f8c64462667c,@roha_43,@mobiinside_637,@kangjagga_113,@beauvoirterry_31,@hanbomsori_6,@haksookim_64,@danielk_4,@littlepeace_1,@kimsawon_42,...,@blueear64_12,@5mindlae_21,@skyscanner_252,@simplelife_43,@ideality_74,@lcs5073_88,@koreatrail_78,@chulw_34,@mymykmcyobv_210,@reading15m_117
7,#157f9687d223501f0e18cc469462cab1,@brunch9uz5_208,@dailylife_230,@hermite236_840,@architect-shlee_993,@cmh102456_6,@bam81054_54,@kwong7_161,@sweetdoraji_143,@seungwhanlee83_26,...,@kiun1018_163,@bookfit_1167,@altna84_25,@jordan777_2109,@garbar_575,@iharu_54,@seanpyo_144,@ilsanpaik_16,@dryjshin_83,@najye0903_1
8,#dfe07d49ef81ac7f00653b5b82b70d83,@jm7654_103,@se7088_1198,@gnsrn66_8,@badventure_287,@kimsawon_3,@mobiinside_916,@keunheo_77,#5459ac2d70cf188ed7923383def0cefc_37,@kostorykr_33,...,@hyun1961_16,@eureka_20,@sluggerpark_27,@omoggy_39,@jordan777_339,@garbar_481,@road4535_46,@yeinvision_13,@jinhahahaha_33,@wander_79
9,#dce6d42815d547618757cd3d963c112c,@ontherecord_59,@oberachu_7,@haksookim_55,@jinyoung5062_244,@dryjshin_200,@kahyun_21,@jay8zg0_176,@99production_8,@ladyjane_3,...,@jbleev_67,@simplebean33_184,@brunch6hjx_19,@deuxji_66,@musestory_4,@fscloud_158,@myungsunkim_32,@bbuni-story_24,@flowerdog_94,@ljs75600_28


결과 저정 끝

아래는 부분 과정 연습을 위한 공간 및 기억을 위해 저장

In [95]:
train_user.tail()

,user_id,user_id_int
2995,#c8bdf5ee13c87360dd14ffea7f63483f,291058
2996,#e8dbed7a4331fb9332d6e9fdf338e67d,278712
2997,#3322441e6904f096c8a86ea5f263bf2a,2852
2998,#8a990bbf90490dfb2a4c4ef8deffc136,7962
2999,#5bc111691522e25777d61b939ca9e3f9,297444


In [102]:
for i in range(3000):
    if recommend_df_copy["user_id"][i] != train_user["user_id"][i]:
        print(i)


In [157]:
net.to("cpu")

NeuralMatrixFactorization(
  (user_emb): Embedding(306222, 100, padding_idx=0)
  (item_emb): Embedding(505841, 100, padding_idx=0)
  (mlp): Sequential(
    (0): Linear(in_features=200, out_features=100, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=100, out_features=100, bias=True)
    (4): ReLU()
    (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Dropout(p=0.5)
    (7): Linear(in_features=100, out_features=1, bias=True)
  )
)

In [43]:
ex_df = train_user.head().copy()

In [81]:
ex_df

,user_id,user_id_int
0,#d6866a498157771069fdf15361cb012b,282354
1,#f963fb8c5d9d14d503fc4e80bd8617b4,31643
2,#87a6479c91e4276374378f1d28eb307c,253506
3,#677e984e245b344f61dc5d3cc1f352c8,170668
4,#519f45eb14e4807e8714fb7e835463eb,150484


In [58]:
ex_df.iloc[0:1]

,user_id,user_id_int
0,#d6866a498157771069fdf15361cb012b,282354


In [117]:
df = pd.DataFrame()
df

""


In [129]:
%%time
for i in ex_df.apply(answering,axis=1):
    s = pd.Series(i)
    df = df.append(s,ignore_index=True)

C:\Users\zone\Anaconda3\lib\site-packages\torch\nn\functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Wall time: 33.3 s


In [130]:
df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,21332.0,178309.0,91359.0,20096.0,53438.0,151353.0,29891.0,29890.0,91356.0,21429.0,...,16412.0,388840.0,362534.0,254808.0,496535.0,16414.0,29301.0,425694.0,95561.0,438187.0
1,95554.0,1168.0,14060.0,95556.0,6808.0,1417.0,65859.0,45403.0,890.0,21221.0,...,365218.0,496564.0,502390.0,418385.0,864.0,113990.0,198225.0,10070.0,445104.0,184779.0
2,17123.0,446355.0,22723.0,73684.0,39766.0,40743.0,8046.0,8682.0,43200.0,65867.0,...,70753.0,179307.0,16399.0,415738.0,452595.0,95565.0,70941.0,95566.0,367893.0,39739.0
3,504318.0,301761.0,307098.0,328419.0,45412.0,1420.0,40551.0,68078.0,459305.0,4756.0,...,91365.0,14067.0,8682.0,134110.0,29895.0,91360.0,91364.0,13965.0,91361.0,21229.0
4,40687.0,95497.0,503572.0,391147.0,184721.0,823.0,65810.0,6084.0,2852.0,40688.0,...,16943.0,390630.0,12119.0,44226.0,223025.0,43382.0,374472.0,442057.0,390632.0,147230.0


In [131]:
ex_df2 = ex_df.join(df)

In [132]:
ex_df2

,user_id,user_id_int,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
0,#d6866a498157771069fdf15361cb012b,282354,21332.0,178309.0,91359.0,20096.0,53438.0,151353.0,29891.0,29890.0,...,16412.0,388840.0,362534.0,254808.0,496535.0,16414.0,29301.0,425694.0,95561.0,438187.0
1,#f963fb8c5d9d14d503fc4e80bd8617b4,31643,95554.0,1168.0,14060.0,95556.0,6808.0,1417.0,65859.0,45403.0,...,365218.0,496564.0,502390.0,418385.0,864.0,113990.0,198225.0,10070.0,445104.0,184779.0
2,#87a6479c91e4276374378f1d28eb307c,253506,17123.0,446355.0,22723.0,73684.0,39766.0,40743.0,8046.0,8682.0,...,70753.0,179307.0,16399.0,415738.0,452595.0,95565.0,70941.0,95566.0,367893.0,39739.0
3,#677e984e245b344f61dc5d3cc1f352c8,170668,504318.0,301761.0,307098.0,328419.0,45412.0,1420.0,40551.0,68078.0,...,91365.0,14067.0,8682.0,134110.0,29895.0,91360.0,91364.0,13965.0,91361.0,21229.0
4,#519f45eb14e4807e8714fb7e835463eb,150484,40687.0,95497.0,503572.0,391147.0,184721.0,823.0,65810.0,6084.0,...,16943.0,390630.0,12119.0,44226.0,223025.0,43382.0,374472.0,442057.0,390632.0,147230.0


In [134]:
ex_df2[0].map(article_dic_swap)

0      @coolfelix_1
1      @basenell_38
2       @brunch_126
3        @99rock_55
4    @addcampus_266
Name: 0, dtype: object

In [127]:
df = pd.DataFrame()
for i in range(3):
    s = pd.Series([1,1])
    df.append(s,ignore_index=True)

In [125]:
s = pd.Series([1,1])
df.append(s,ignore_index=True)

,0,1
0,1.0,1.0


In [58]:
query2 = torch.stack([
    torch.zeros(max_item+1, dtype=torch.int64).fill_(7385),
    torch.arange(0, max_item+1)
], 1).long()

In [59]:
scores, indices = torch.topk(net(query2.to("cuda:0")), 100)

C:\Users\zone\Anaconda3\lib\site-packages\torch\nn\functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [49]:
indices

tensor([  5440, 428829,  73685, 438203, 127661,  28161,  16399, 501088, 142847,
        458946,   1420,  57935, 355221,  19727,  29894,   8682,   1998, 365543,
        250909, 161399, 432212, 477524,  16415, 331930,  91360, 162666, 327854,
         42953, 467981, 357116, 453143, 499933, 415186, 358699,  42015, 505121,
        399716, 171943, 326826,  80672,  12159, 405881, 504082, 163115,  77286,
        356362, 403032, 344219, 363929, 329143,  51144,  70943, 369789, 420005,
        452032, 425533, 393712, 271593,  72925, 148333, 500490,   1774,  29814,
        338231, 362598, 435179, 101982, 379758, 205118, 100967, 485562, 127663,
        247437,  17869, 218152,   7134,  35043, 332971,   7791, 386297,  51345,
        498909,  40742,  70206,  29895, 301874, 360535, 390016, 458645,  67924,
        317452,  30967, 474553, 248291, 315274, 457956,  13436, 315325, 399107,
        437114], device='cuda:0')

In [37]:
scores

tensor([5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000,
        5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000,
        5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000,
        5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000,
        5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000,
        4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999,
        4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999,
        4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999,
        4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999,
        4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999,
        4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9999,
        4.9999], device='cuda:0', grad_fn=<TopkBackward>)

In [53]:
torch.save(net.state_dict(),'model.pth')

In [57]:
model = NeuralMatrixFactorization(max_user+1, max_item+1)
model.load_state_dict(torch.load('model.pth'))
model.eval()

RuntimeError: Error(s) in loading state_dict for NeuralMatrixFactorization:
	size mismatch for user_emb.weight: copying a param with shape torch.Size([306222, 10]) from checkpoint, the shape in current model is torch.Size([306222, 15]).
	size mismatch for item_emb.weight: copying a param with shape torch.Size([505841, 10]) from checkpoint, the shape in current model is torch.Size([505841, 15]).
	size mismatch for mlp.0.weight: copying a param with shape torch.Size([50, 20]) from checkpoint, the shape in current model is torch.Size([50, 30]).

In [149]:
model.to("cpu")

NeuralMatrixFactorization(
  (user_emb): Embedding(306222, 100, padding_idx=0)
  (item_emb): Embedding(505841, 100, padding_idx=0)
  (mlp): Sequential(
    (0): Linear(in_features=200, out_features=100, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=100, out_features=100, bias=True)
    (4): ReLU()
    (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Dropout(p=0.5)
    (7): Linear(in_features=100, out_features=1, bias=True)
  )
)

In [169]:
scores, indices2 = torch.topk(model(query2.to("cpu")), 100)

In [170]:
indices2

tensor([503777,  40741,  35044,  16415,   1171,  43437,  70271,  44281,  19727,
         37128,  70942,  91360,  40742,  21335,   3194,  33632,  44654,  19030,
          1420, 438203,  68144,  70943,   8682, 448901,  19728,  46557,   7791,
           826,  73685,  10077,   2868,  13965,  17123,   1774,  22194,  70944,
         95566,  50075,  21229,  21432,  20720,  70945, 358699,  42043,  16985,
         45412,  95568,  65868, 350537,    893,  31981,  95567,  16399,  29894,
         32993,  91361, 250909,  10498,  10422,  11739, 391246,   6106, 361137,
         18776,  37852, 446260,   1998,  82320,   6812,  14986, 480367, 151361,
         35043,  70754,  68078,  39739,  53442,  57286,  40743,  13684,  57935,
         14067, 235798,   2521, 114002,  16417, 442607, 317251, 134111,   5440,
        307106,   5441, 427051, 179308,    896,  44653,  91359,   8046, 483202,
         11130])

In [171]:
scores

tensor([4.9999, 4.9999, 4.9999, 4.9999, 4.9999, 4.9998, 4.9998, 4.9998, 4.9998,
        4.9997, 4.9997, 4.9997, 4.9997, 4.9996, 4.9996, 4.9996, 4.9995, 4.9995,
        4.9994, 4.9993, 4.9993, 4.9993, 4.9993, 4.9993, 4.9993, 4.9993, 4.9992,
        4.9992, 4.9992, 4.9992, 4.9992, 4.9992, 4.9991, 4.9991, 4.9991, 4.9991,
        4.9990, 4.9990, 4.9989, 4.9988, 4.9987, 4.9987, 4.9987, 4.9987, 4.9986,
        4.9986, 4.9986, 4.9985, 4.9985, 4.9984, 4.9984, 4.9984, 4.9982, 4.9982,
        4.9981, 4.9981, 4.9980, 4.9980, 4.9979, 4.9979, 4.9979, 4.9978, 4.9978,
        4.9978, 4.9976, 4.9976, 4.9976, 4.9974, 4.9974, 4.9973, 4.9973, 4.9973,
        4.9973, 4.9972, 4.9972, 4.9969, 4.9969, 4.9969, 4.9969, 4.9969, 4.9967,
        4.9967, 4.9967, 4.9966, 4.9966, 4.9966, 4.9966, 4.9965, 4.9964, 4.9963,
        4.9963, 4.9963, 4.9962, 4.9961, 4.9961, 4.9960, 4.9959, 4.9959, 4.9957,
        4.9956], grad_fn=<TopkBackward>)

In [168]:
model2 = NeuralMatrixFactorization()
model2.load_state_dict(torch.load('model.pth'))
model2.eval()

TypeError: __init__() missing 2 required positional arguments: 'max_user' and 'max_item'

In [35]:
embedding = torch.nn.Embedding(3000,10,0)
ie = torch.LongTensor([[1,2,4,5],[4,3,2,9]])
embedding(ie)

tensor([[[-2.9979e-01, -2.6795e-02, -3.4787e-01,  1.2488e+00, -8.8538e-01,
          -9.5639e-02, -9.1484e-01, -2.7490e-01,  1.5418e+00, -3.3401e-01],
         [ 2.1458e-01,  1.0407e-03, -1.7728e+00,  1.2471e+00,  9.4691e-01,
           1.2407e+00,  7.4261e-01,  5.5209e-01,  1.3386e+00, -1.3531e-01],
         [ 1.5223e+00,  9.3405e-01, -1.0017e+00,  5.3221e-01,  1.3854e+00,
           1.0754e+00,  8.4165e-01, -2.6806e-01,  1.2123e-01,  1.7769e+00],
         [-1.2388e+00,  1.0264e-01, -1.0211e+00,  1.2616e+00, -2.1533e+00,
           6.9840e-01,  2.1033e-01,  7.4009e-01,  7.7347e-01, -6.8579e-02]],

        [[ 1.5223e+00,  9.3405e-01, -1.0017e+00,  5.3221e-01,  1.3854e+00,
           1.0754e+00,  8.4165e-01, -2.6806e-01,  1.2123e-01,  1.7769e+00],
         [ 1.6428e+00, -6.4701e-01,  3.3597e-01, -8.4460e-01,  2.1505e+00,
           1.0453e+00, -9.7649e-02, -7.4788e-01, -1.1477e+00,  2.3284e+00],
         [ 2.1458e-01,  1.0407e-03, -1.7728e+00,  1.2471e+00,  9.4691e-01,
           1.2407